%matplotlib inline

In [1]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [2]:
import tqdm
import time

In [3]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name

log_file_name='01_21_18:38:20.log'


In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [5]:
eval_df = pd.read_csv('../../data/Manually_Annotated_file_lists/validation_face_mesh_crop.csv')
eval_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + eval_df.subDirectory_filePath


eval_df = eval_df[eval_df['have_facemesh']]

In [6]:
batch_size = 8
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()
eval_logs = []

In [7]:
import os
model_files = os.listdir("./models")
model_files.remove('.ipynb_checkpoints')

In [8]:
len(model_files)

87

In [9]:
model_files.sort()

In [10]:
eval_result_df = pd.read_csv('./eval_df.csv')

In [11]:
model_files = list(filter(lambda x: x not in list(eval_result_df.model), model_files))

In [ ]:
model_files

In [16]:
model = tf.keras.models.load_model('./models/mobilenet_from_pretrain_no_train/')

total_process = 0
start_time = time.time()

total_loss = 0
total_step = 0
total_valence_loss = 0
total_arousal_loss = 0
total_emotion_loss = 0
total_emotion_correct = 0
for step, row in tqdm.tqdm(enumerate(DataFrameBatchIterator(eval_df, batch_size=batch_size)), total = len(eval_df) / batch_size):
    if len(row) == 0:
        continue
    imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
        cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

    img_array = np.array(list(imgs))


    y_valence = np.array(row.valence)
    y_arousal = np.array(row.arousal)
    y_emotion = np.array(row.expression)


    logits = model(img_array, training=True)

    if model.output_shape != (None, 11):
        pred_valence = logits[0]
        pred_arousal = logits[1]
        pred_emotion = logits[2]
        break
        valence_loss = MSE_loss(y_valence, pred_valence)
        arousal_loss = MSE_loss(y_arousal, pred_arousal)
    else:
        valence_loss = 0
        arousal_loss = 0
        pred_emotion = logits


    emotion_loss = SCC_loss(y_emotion, pred_emotion)

    loss = valence_loss + arousal_loss + emotion_loss

    total_loss += float(loss)
    total_step += 1
    total_process += len(img_array)

    total_valence_loss += float(valence_loss)
    total_arousal_loss += float(arousal_loss)
    total_emotion_loss += float(emotion_loss)

    emotion_correct = pred_emotion.numpy().argmax(axis = 1) == y_emotion
    total_emotion_correct += sum(emotion_correct)

#lprint(f"--- {time.time() - start_time} seconds for evaluate {total_process} image ---\n")
eval_logs.append({
    'model': model_file,
    'total_emotion_correct': total_emotion_correct,
    'total_loss': total_loss,
    'total_valence_loss': total_valence_loss,
    'total_arousal_loss': total_arousal_loss,
    'total_emotion_loss': total_emotion_loss,
    'total_process': total_process,
    'time_take': time.time() - start_time
})


  0%|                                                 | 0/681.5 [00:00<?, ?it/s]


SyntaxError: 'break' outside loop (265838384.py, line 67)

In [15]:
pred_valence

NameError: name 'pred_valence' is not defined

In [ ]:
df = pd.DataFrame(eval_logs)